In [1]:
import pandas as pd
import numpy as np

In [121]:
from sklearn.model_selection import train_test_split

In [8]:
data = pd.read_csv('../data/data.csv')

In [9]:
data.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [11]:
(data['comment_text'] == ' ').any()

False

In [12]:
data.isnull().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [13]:
data['comment_text'].fillna(value = '_na_',inplace=True)

In [26]:
vocabulary = [pair[0] for pair in word_counter.most_common(vocab_size)]

In [27]:
glove_dim = len(embeddings)

In [103]:
nb_words = min(vocab_size+1, glove_dim)
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for i, word in enumerate(vocabulary):
    if i >= vocab_size+1:
        continue
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i+1] = embedding_vector

In [105]:
embedding_matrix[0]

array([-0.90863209, -1.83054233,  0.20075362, -0.09392044, -0.61499785,
       -0.08694068, -0.53790446, -0.69105299,  0.24632631,  1.114145  ,
        0.20596222,  0.1127703 ,  0.1055719 , -0.49976979,  0.16715639,
       -0.61481833,  0.5863114 , -0.59645605, -0.90389832, -0.22466315,
       -0.40626646, -0.65643533,  0.32985416, -0.09072985,  1.4024442 ,
        0.79602253, -1.16494341, -0.12527208, -0.07377362,  0.02483354,
        0.60662275,  0.07979284,  1.02550161, -0.02989132,  0.38192396,
       -1.15426754, -0.14828249,  1.46536808,  0.11470578,  0.80675531,
        0.05579536,  0.89502589, -0.87188143,  0.84338568,  1.14738712,
        0.06312929, -0.85381121,  0.14439408, -0.15484467, -0.46886253])

In [100]:
idx_to_word = {index+1:word for index,word in enumerate(vocabulary)}
word_to_idx = {word:index+1 for index,word in enumerate(vocabulary)}
idx_to_word[0] = '<pad>'
word_to_idx['<pad>'] = 0

In [101]:
def make_seq(df):
    all_tokens = [spacy_tokenizer(df["comment_text"].iloc[i]) for i in range(df["comment_text"].shape[0])]
    seq_all = []
    empty_indices = []
    for i in range(df.shape[0]):
        seq = []
        for tok in all_tokens[i]:
            tok_idx = word_to_idx.get(tok)
            if tok_idx is None:
                continue
            seq.append(tok_idx)
        if(len(seq) == 0):
            empty_indices.append(i)
            continue
        seq_all.append(seq)
        
    target_df = df.loc[~df.index.isin(empty_indices), df.columns[2:]]
    target = target_df.as_matrix()
        #print(tok)
    return seq_all,target

In [102]:
all_seq,target = make_seq(train_data)

In [106]:
max_len = 100

In [107]:
def pad_sequences(vectorized_seqs, seq_lengths):
    max_seq_len = min(seq_lengths.max(),max_len)
    seq_tensor = torch.zeros((len(vectorized_seqs), max_seq_len)).long()
    for idx, (seq, seqlen) in enumerate(zip(vectorized_seqs, seq_lengths)):
        min_seq_len = min(seqlen,max_len)
        seq_tensor[idx, :min_seq_len] = torch.LongTensor(seq[:min_seq_len])
    return seq_tensor

In [108]:
all_seq_lengths = torch.LongTensor([len(s) for s in all_seq])

In [109]:
all_seq_pad = pad_sequences(all_seq,all_seq_lengths)

In [162]:
x_train,x_val,y_train,y_val = train_test_split(pd.DataFrame(all_seq_pad.numpy()),pd.DataFrame(target),test_size = 0.2,stratify = target[:,3]) 

In [135]:
y_train.sum()

27607

In [178]:
train_ids = torch.from_numpy(x_train.index.values).long()
val_ids = torch.from_numpy(x_val.index.values).long()

In [179]:
train_seq_lengths = torch.index_select(all_seq_lengths,0,train_ids)
val_seq_lengths = torch.index_select(all_seq_lengths,0,val_ids)

In [142]:
y_train[:,5].sum()/target[:,5].sum()

0.82583454281567492

In [147]:
batch_size=64

In [161]:
class ToxicDataset(Dataset):
    def __init__(self,data_tensor,target_tensor,length_tensor):
        assert data_tensor.size(0) == target_tensor.size(0) == length_tensor.size(0)
        self.data_tensor = data_tensor
        self.target_tensor = target_tensor
        self.length_tensor = length_tensor
    
    def __getitem__(self,index):
        return self.data_tensor[index],self.target_tensor[index],self.length_tensor[index]
    
    def __len__(self):
        return self.data_tensor.size(0)

In [180]:
x_train_t = torch.from_numpy(x_train.values).long()
y_train_t = torch.from_numpy(y_train.values).long()
train_dataset = ToxicDataset(x_train_t,y_train_t,train_seq_lengths)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True,num_workers=2)

In [182]:
x_val_t = torch.from_numpy(x_val.values).long()
y_val_t = torch.from_numpy(y_val.values).long()
val_dataset = ToxicDataset(x_val_t,y_val_t,val_seq_lengths)
val_dataloader = DataLoader(val_dataset, batch_size=2*batch_size,shuffle=False,num_workers=2)

In [149]:
embed_dim = 50

In [152]:
embed_nn = nn.Embedding(vocab_size+1, embedding_dim=embed_dim)

In [153]:
embed_nn.weight.data.copy_(torch.from_numpy(embedding_matrix))


-9.0863e-01 -1.8305e+00  2.0075e-01  ...   1.4439e-01 -1.5484e-01 -4.6886e-01
 4.9427e-01  1.3234e-01 -2.3199e-02  ...   2.4152e-01 -2.7658e-01 -3.7987e-01
 1.2900e+00  5.2437e-02 -4.3540e-01  ...   4.4992e-02 -9.1239e-01 -1.0489e+00
                ...                   ⋱                   ...                
-2.7085e-01 -9.7770e-01 -9.7691e-01  ...   7.1800e-01  4.4413e-01 -4.4221e-01
 1.1607e-01  7.2430e-01 -5.0700e-01  ...   3.4732e-01 -1.0340e+00 -6.5862e-01
 1.1510e+00  8.3473e-02 -2.0625e-01  ...   1.2821e+00  5.4254e-02 -1.1215e-02
[torch.FloatTensor of size 1001x50]

In [154]:
embed_nn.weight.requires_grad = False

In [156]:
epochs = 1

In [183]:
class LSTMBasicNet(nn.Module):
    def __init__(self,input_size, output_size,hidden_size,num_layers,num_classes,embed):
        super(LSTMBasicNet,self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.num_layers = num_layers
        self.embed = embed
        self.lstm = nn.LSTM(input_size=self.input_size,hidden_size=hidden_size,num_layers = num_layers,bidirectional = True, batch_first = True, dropout = 0.1)
        self.fc1 = nn.Linear(in_features=hidden_size*2,out_features=output_size)
        self.fc2 = nn.Linear(in_features=output_size,out_features=num_classes)
        
    def forward(self,x,lengths):
        embed_out = self.embed(x)
        packed_input = pack_padded_sequence(embed_out,batch_first=True,lengths=lengths) 
        h0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)) # 2 for bidirection 
        c0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size))
        packed_output,(hn,cn) = self.lstm(packed_input,(h0,c0))
        lstm_out = pad_packed_sequence(packed_output,batch_first=True)
        lstm_out = F.tanh(lstm_out)
        fc1_out = F.dropout(F.relu(self.fc1(lstm_out[:,-1,:])),p=0.1)
        out = F.sigmoid(self.fc2(fc1_out))
        
        return out
        
        
        
        

In [184]:
model = LSTMBasicNet(embed_size,64,64,2,6,embed_nn)